In [5]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import pickle

%matplotlib widget

### Bias analysis

In [6]:
bias_list = np.genfromtxt('../group08_HAT-P-12_20230214/bias/bias_list.txt', dtype=str)
bias_fits = fits.open('../group08_HAT-P-12_20230214/bias/'+bias_list[0]) #takes the first bias .fits file
bias_hdu = bias_fits[0]
bias_hdu.header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  521 / length of data axis 1                          
NAXIS2  =                  156 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BZERO   =                32768 / offset data range to that of unsigned short    
BSCALE  =                    1 / default scaling factor                         
DATE    = '2023-02-15T05:14:25' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
FILENAME= 'AF824296.fits'      / Original file name                             
TIMESYS = 'UTC     '        

In [7]:
bias_time = bias_hdu.header['JD']
bias_airmass = bias_hdu.header['AIRMASS']
bias_gain = bias_hdu.header['GAIN']
bias_gain_comment = bias_hdu.header.comments['GAIN'] #unit
bias_readout_noise = bias_hdu.header['RDNOISE']
bias_ron_comment = bias_hdu.header.comments['RDNOISE']

print('Julian date : {0:12.6f} JD'.format(bias_time)) #converted to UTC: 2023-02-15 05:14:25.094
print('CCD Gain : {0:3.2f} {1:.7s}'.format(bias_gain, bias_gain_comment)) #converts ADU in electrons (WATCH OUT! some instruments have an opposite definition, i.e. [ADU/e])
print('CCD Readout noise: {0:3.2f} {1:.3s}'.format(bias_readout_noise, bias_ron_comment))

bias_data = bias_hdu.data * bias_gain   #converted to electrons

Julian date : 2459990.718346 JD
CCD Gain : 1.91 [e/ADU]
CCD Readout noise: 7.10 [e]


In [8]:
n_images = len(bias_list) #we check the number of frames since we might have commented the original file because some frames may be damaged or not good for the analysis
n_images

30

In [ ]:
dim00, dim01 = np.shape(bias_data)
stack = np.empty([n_images, dim00, dim01])

for i, name in enumerate(bias_list):

    bias_temp = fits.open('../group08_HAT-P-12_20230214/bias/' + name)
    stack[i, :, :] = bias_temp[0].data * bias_temp[0].header['GAIN']    #we saved the bias data multiplied by the gain (to convert into electrons) in the stack array
    bias_temp.close()

median_bias = np.median(stack, axis = 0)

In [ ]:
print(np.amin(bias_data))
print(np.amax(bias_data))

In [ ]:
fig, ax = plt.subplots(2, figsize = (10, 6))

im1 = ax[0].imshow(bias_data, vmin = 2770, vmax = 2810, origin='lower')
im2 = ax[1].imshow(median_bias, vmin = 2770, vmax = 2810, origin='lower')
cbar = fig.colorbar(im1, ax=ax)
cbar.set_label('e')
ax[0].set_xlabel('X [pixel]')
ax[0].set_ylabel('Y [pixel]')
ax[1].set_xlabel('X [pixel]')
ax[1].set_ylabel('Y [pixel]')
plt.show()

In [ ]:
fullframe_error_median_bias = np.std(stack, axis=0)/np.sqrt(np.shape(stack)[0])

In [ ]:
pickle.dump(median_bias, open('../Results/median_bias.p', 'wb'))   #wb: write bite / saves the values in a new file
pickle.dump(np.median(fullframe_error_median_bias), open('../Results/median_bias_error.p', 'wb')) 

### Flat analysis

In [ ]:
flat_list = np.genfromtxt('../group08_HAT-P-12_20230214/flat/flat_list.txt', dtype=str)
n_flat = len(flat_list)

In [ ]:
median_bias = pickle.load(open('../Results/median_bias.p', 'rb'))
median_bias_error = pickle.load(open('../Results/median_bias_error.p', 'rb'))

flat00_fits = fits.open('../group08_HAT-P-12_20230214/flat/' + flat_list[0])
flat00_gain = flat00_fits[0].header['GAIN']
flat00_ron = flat00_fits[0].header['RDNOISE']
flat00_data = flat00_fits[0].data * flat00_gain

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 6))
im1 = ax[0].imshow(flat00_data[:, 20:490], origin='lower', cmap='magma')   #the indexing is to exclude the black columns in the CCD
median_column = np.average(flat00_data[:, 20:490], axis=0)
im2 = ax[1].plot(median_column)
cbar = fig.colorbar(im1, ax=ax)

cbar.set_label('e')
ax[0].set_xlabel('X [pixels]')
ax[0].set_ylabel('Y [pixels]')
ax[1].set_xlabel('X [pixels]')
ax[1].set_ylabel('Average counts')
plt.show()

In [ ]:
flat_dim00, flat_dim01 = np.shape(flat00_data)
stack = np.empty([n_flat, flat_dim00, flat_dim01])

for i_flat, flat_name in enumerate(flat_list):
    flat_temp = fits.open('../group08_HAT-P-12_20230214/flat/' + flat_name)
    stack[i_flat, :, :] = flat_temp[0].data * flat_temp[0].header['GAIN'] - median_bias #we remove the bias to know the real value of the photons reaching the CCD
    flat_temp.close()

In [ ]:
window_size = 50

x0 = np.int16(flat_dim01/2 - window_size/2)
x1 = np.int16(flat_dim01/2 + window_size/2)

y0 = np.int16(flat_dim00/2 - window_size/2)
y1 = np.int16(flat_dim00/2 + window_size/2)

flat_selection_median = np.median(flat00_data[y0:y1, x0:x1])

In [ ]:
normalization_factor = np.median(stack[:,  y0:y1, x0:x1], axis=(1, 2))
print(normalization_factor)

In [ ]:
x_axis = np.arange(0, 30, 1)
plt.figure(figsize=(8, 4))
plt.scatter(x_axis, normalization_factor)
plt.show()

In [ ]:
stack_normalized_iter = np.zeros_like(stack)

for i_flat in range(0, n_flat):
    stack_normalized_iter[i_flat, :, :] = stack[i_flat, :, :]/normalization_factor[i_flat]

median_normalized_flat = np.median(stack_normalized_iter, axis = 0)

In [ ]:
stack_associated_error = np.sqrt(stack + median_bias_error**2 + flat00_ron**2)
normalized_flat_error = (stack_associated_error.T / normalization_factor).T
median_normalized_flat_error = np.sqrt(np.sum(normalized_flat_error**2, axis=0) / n_flat**2 )

In [ ]:
pickle.dump(median_normalized_flat, open('../Results/median_normalized_flat.p', 'wb'))
pickle.dump(median_normalized_flat_error, open('../Results/median_normalized_flat_error.p', 'wb'))